# 概要
* llama_indexのPromptHelperを触ってみる

# 参考
* トークンの制約に対応するための汎用プロンプトサポートツール
* https://gpt-index.readthedocs.io/en/latest/reference/prompt_helper.html

In [1]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
#Jupyterでも実行できるのは知らんかった...
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

# PromptHelperとは
* トークンの制約に対応するための汎用プロンプトサポートツール

# LLM（Large Language Model）
* PromptHelperの引数の大部分はLLMに関連する
* つまり最初にLLMが何かを理解する必要がある
* (そもそも入力されたPromptをparseするのはLLMなのでPromptHelperがLLMを引数にとるのは当然である)
* 詳細はここを見よ！
* https://gpt-index.readthedocs.io/en/latest/reference/llm_predictor.html

# そもそも
* LLMPredictorとはLangchainのLLMChainのラッパーであり、LlamdaIndexと統合することができる
* LLMChain ↔ LLMPrecitor ↔ LlamdaIndex
* Our LLMPredictor is a wrapper around Langchain’s LLMChain that allows easy integration into LlamaIndex.
* llmを引数にとり、デフォルトではOpenAI’s text-davinci-003が指定される
* max_input_size: int = 3900, num_output: int = 256
* デフォルトでは入力は3900トークンで出力は256トークン
* つまり長文の回答はできない

# 引数のllm
* OpenAI以外にいも複数のLLMをサポートしている
* OpenAI, Cohere, Hugging Face, etc
* Llama_indexは自身でLLMはサポートしておらず、あくまでもlangchainを介して呼び出しを行う？

In [3]:
from gpt_index import LLMPredictor, GPTSimpleVectorIndex, SimpleDirectoryReader

In [4]:
# OpenAIのAPIのラッパー
from langchain.llms import OpenAI

# 利用例

In [41]:
from llama_index import (
    GPTSimpleVectorIndex, 
    GPTSimpleKeywordTableIndex, 
    GPTListIndex, 
    SimpleDirectoryReader
)

from llama_index import (
    GPTKeywordTableIndex, 
    SimpleDirectoryReader, 
    LLMPredictor,
    PromptHelper
)

from gpt_index.indices.knowledge_graph.base import GPTKnowledgeGraphIndex
from langchain import OpenAI

documents = SimpleDirectoryReader('story/dragonball').load_data()

# define prompt helper
# set maximum input size
max_input_size = 4096
# set number of output tokens
num_output = 2048
# set maximum chunk overlap
max_chunk_overlap = 0
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap, separator="\n\n", chunk_size_limit=512)

# define LLM
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=num_output))

# build index
index = GPTSimpleVectorIndex(documents, llm_predictor=llm_predictor, prompt_helper=prompt_helper)

DEBUG:root:> [SimpleDirectoryReader] Total files added: 1
DEBUG:root:> Adding chunk: 孫悟空少年編
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た...
DEBUG:root:> Adding chunk: その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月...
DEBUG:root:> Adding chunk: ピッコロ大魔王編
天下一武道会終了後、ピラフ一味によって復活したピッコロ大魔王によって、クリリ...
DEBUG:root:> Adding chunk: その約3年後、少年から青年へと成長した悟空は、天下一武道会の会場で仲間たちと再会。試合では、少...
DEBUG:root:> Adding chunk: サイヤ人編
ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごし...
DEBUG:root:> Adding chunk: 悟空はドラゴンボールによって生き返るまでの間、あの世の界王の下で修業し、仲間と共に地球に強襲し...
DEBUG:root:> Adding chunk: フリーザ編
地球の神と殺された仲間たちを甦らせるため、重傷で入院中の悟空に代わり、悟飯、クリリ...
DEBUG:root:> Adding chunk: 出し抜かれて願いが叶えられなかったフリーザは激怒し、一行は対決を強いられる。フリーザの持つ圧倒...
DEBUG:root:> Adding chunk: 人造人間・セル編
ナメック星での闘いから約1年後、密かに生き延びていたフリーザとその一味が地球...
DEBUG:root:> Adding chunk: 人造人間17号と人造人間18号を吸収することで完全体となったセルは地球の命運を賭けた武道会「セ...
DEBUG:root:> Adding chunk: 魔人ブウ編
セルゲームより約7年後、高校生に成長した悟飯が天下一武道会に出場することを知った悟...
DEBUG:root:> Adding chunk: 一方、魔人ブウは様々な人間との出会いからより邪悪で強力な魔人へと変貌。悟天とトランクスが「フュ...
DEBUG:

DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 82889
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=116 request_id=b192b22d7d2af0dcf73f3914ea132a3d response_code=200
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u5b6b\\u609f\\u7a7a\\u5c11\\u5e74\\u7de8 \\u5730\\u7403\\u306e\\u4eba\\u91cc\\u96e2\\u308c\\u305f\\u5c71\\u5965\\u306b\\u4f4f\\u3080\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u30fb\\u5b6b\\u609f\\u7a7a\\u306f\\u3042\\u308b\\u65e5\\u3001\\u897f\\u306e\\u90fd\\u304b\\u3089\\u3084\\u3063\\u3066\\u6765\\u305f\\u5c11\\u5973\\u30fb\\u30d6\\u30eb\\u30de\\u3068\\u51fa\\u4f1a\\u3046\\u3002\\u305d\\u3053\\u3067\\u

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 99439
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=191 request_id=956a61eeac650502eb4cf2f355c5e486 response_code=200
INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 4358 tokens


In [42]:
with open('story/dragonball/dragonball.txt') as f:
    s = f.read()
    #print(type(s))
    print(s)

孫悟空少年編
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。

その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。

ピッコロ大魔王編
天下一武道会終了後、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。闘いの後、悟空はピッコロ大魔王に殺された神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。そこで神龍復活の条件として、神の下で天界で修行することとなった。

その約3年後、少年から青年へと成長した悟空は、天下一武道会の会場で仲間たちと再会。試合では、少年時代に出会った際に「嫁にもらう」と約束していた牛魔王の娘・チチと再会しその場で結婚。そして武道会に出場していたピッコロ大魔王の生まれ変わりであるマジュニアと決勝戦で激突、これに勝利し初の天下一武道会優勝を飾る。

サイヤ人編
ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごしてい

In [43]:
doclist = (s.split("\n\n"))

In [21]:
len(doclist)

14

In [13]:
index.query("ドラゴンボールのあらすじを要約してください")

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u306e\\u3042\\u3089\\u3059\\u3058\\u3092\\u8981\\u7d04\\u3057\\u3066\\u304f\\u3060\\u3055\\u3044"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 8419
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=16 request_id=33aa2089856ffd9d022479344f9daf66 response_code=200
DEBUG:llama_index.indices.utils:> Top 1 nodes:
> [Node 20230f7d-ae5e-4731-8692-e8e81b390e69] [Similarity score:             0.86637] 孫悟空少年編
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれ...
DEBUG:root:> Searching in chunk: 孫悟空少年編
地球の人里離れた山奥に住む尻尾の生えた少年・

Response(response='\n孫悟空とブルマは、7つのドラゴンボールを集めることで神龍を呼び出すことができることを知り、それらを探す旅に出る。しかし、ピラフ一味がドラゴンボールを奪って神龍を呼び出そうとするが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。', source_nodes=[SourceNode(source_text='孫悟空少年編\n地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。', doc_id='e360e30e-6706-44db-914b-4ecc79916610', extra_info=None, node_info={'start': 0, 'end': 297}, similarity=0.8663698010641532)], extra_info=None)

In [15]:
# うーん、なにか色々混ざっている...
index.query("ドラゴンボールのあらすじを要約してください", similarity_top_k=3)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u306e\\u3042\\u3089\\u3059\\u3058\\u3092\\u8981\\u7d04\\u3057\\u3066\\u304f\\u3060\\u3055\\u3044"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 8419
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=25 request_id=ac60e028eb5aee8a8cb1b7031e8a4e6f response_code=200
DEBUG:llama_index.indices.utils:> Top 3 nodes:
> [Node 20230f7d-ae5e-4731-8692-e8e81b390e69] [Similarity score:             0.86637] 孫悟空少年編
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれ...
> [Node 8d01cdec-a10e-4f81-be43-63c4c67f32cf] [Similarity scor

DEBUG:root:

孫悟空とブルマは、7つのドラゴンボールを集めることで神龍を呼び出すことができることを知り、それらを探す旅に出る。悟空はドラゴンボールによって生き返るまでの間、あの世の界王の下で修業し、仲間と共に地球に強襲したサイヤ人の戦士・ナッパと王子・ベジータを迎え撃つ。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘。仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。しかし、ピラフ一味がドラゴンボールを奪って神龍を呼び出そうとするが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。
DEBUG:root:> Refined response: 

孫悟空とブルマは、7つのドラゴンボールを集めることで神龍を呼び出すことができることを知り、それらを探す旅に出る。悟空はドラゴンボールによって生き返るまでの間、あの世の界王の下で修業し、仲間と共に地球に強襲したサイヤ人の戦士・ナッパと王子・ベジータを迎え撃つ。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘。仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。しかし、ピラフ一味がドラゴンボールを奪って神龍を呼び出そうとするが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。
DEBUG:root:> Refine context: サイヤ人編
ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごし...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["The original question is as follows: \\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u306e\\u3042\\u

Response(response='\n\n孫悟空とブルマは、7つのドラゴンボールを集めることで神龍を呼び出すことができることを知り、それらを探す旅に出る。悟空は宿敵ピッコロと手を組み、自らの命と引き換えに実兄・ラディッツを倒す。その後、約1年後にはさらに強力なサイヤ人たちがドラゴンボールを求めて地球に来襲し、悟空はドラゴンボールによって生き返るまでの間、あの世の界王の下で修業し、仲間と共に地球に強襲したサイヤ人の戦士・ナッパと王子・ベジータを迎え撃つ。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘。仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。しかし、ピラフ一味がドラゴンボールを奪って神龍を呼び出そうとするが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。', source_nodes=[SourceNode(source_text='孫悟空少年編\n地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。', doc_id='e360e30e-6706-44db-914b-4ecc79916610', extra_info=None, node_info={'start': 0, 'end': 297}, similarity=0.8663698010641532), SourceNode(source_text='悟空はドラゴンボールによって生き返るまでの間、あの世の界王の下で修業し、仲間と共に地球に強襲したサイヤ人の戦士・ナッパと王子・ベジータを迎え撃つ。悟空は修行により増した力でナッパを一

In [22]:
# うーん、なにか色々混ざっている...
# 順番が狂っとる
index.query("ドラゴンボールのあらすじを要約してください", similarity_top_k=14)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u306e\\u3042\\u3089\\u3059\\u3058\\u3092\\u8981\\u7d04\\u3057\\u3066\\u304f\\u3060\\u3055\\u3044"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 8419
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=728 request_id=1b21f431a1239db723652cc55971ee71 response_code=200
DEBUG:llama_index.indices.utils:> Top 12 nodes:
> [Node 20230f7d-ae5e-4731-8692-e8e81b390e69] [Similarity score:             0.866381] 孫悟空少年編
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願い

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=20843 request_id=da5effe6796d69190175e5c640df145e response_code=200
DEBUG:root:

孫悟空とブルマは、7つのドラゴンボールを集めることで神龍を呼び出すことができることを知り、それらを探す旅に出る。悟空はドラゴンボールによって生き返るまでの間、あの世の界王の下で修業し、仲間と共に地球に強襲したサイヤ人の戦士・ナッパと王子・ベジータを迎え撃つ。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘。仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。しかし、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで、ピラフ一味がドラゴンボールを奪って神龍を呼び出そうとする野望を阻止する。
DEBUG:root:> Refined response: 

孫悟空とブルマは、7つのドラゴンボールを集めることで神龍を呼び出すことができることを知り、それらを探す旅に出る。悟空はドラゴンボールによって生き返るまでの間、あの世の界王の下で修業し、仲間と共に地球に強襲したサイヤ人の戦士・ナッパと王子・ベジータを迎え撃つ。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘。仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。しかし、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで、ピラフ一味がドラゴンボールを奪って神龍を呼び出そうとする野望を阻止する。
DEBUG:root:> Refine context: サイヤ人編
ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ご

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=30639 request_id=98fffc421fc0382705f99685410b097e response_code=200
DEBUG:root:

孫悟空とブルマは、実兄のラディッツから7つのドラゴンボールを集めることで神龍を呼び出すことができることを知り、それらを探す旅に出る。悟空はドラゴンボールによって生き返るまでの間、あの世の界王の下で修業し、仲間と共に地球に強襲したサイヤ人の戦士・ナッパと王子・ベジータを迎え撃つ。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘。仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。その後、悟飯、クリリン、ブルマの3人が神とピッコロの故郷であるナメック星へ向かい、フリーザとその一味が不老不死を求めて来襲し、ドラゴンボールを略奪しようとしていた。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空とナメック星人たちの協力を得てナメック星の神龍・ポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる。ウーロンがとっさに言い放った下らない願いを叶えてもらうことで、ピラフ一味がドラゴンボールを奪って神龍を呼び出そうとする野望を阻止する。
DEBUG:root:> Refined response: 

孫悟空とブルマは、実兄のラディッツから7つのドラゴンボールを集めることで神龍を呼び出すことができることを知り、それらを探す旅に出る。悟空はドラゴンボールによって生き返るまでの間、あの世の界王の下で修業し、仲間と共に地球に強襲したサイヤ人の戦士・ナッパと王子・ベジータを迎え撃つ。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘。仲間の協力もあり、何とか辛勝し撤退させるが、多

DEBUG:root:> Refined response: 

孫悟空とブルマは、実兄のラディッツから7つのドラゴンボールを集めることで神龍を呼び出すことができることを知り、それらを探す旅に出る。悟空はドラゴンボールによって生き返るまでの間、あの世の界王の下で修業し、仲間と共に地球に強襲したサイヤ人の戦士・ナッパと王子・ベジータを迎え撃つ。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘。仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。その後、悟空は武術の達人・亀仙人の下で8か月間の修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟飯、クリリン、ブルマの3人が神とピッコロの故郷であるナメック星へ向かい、フリーザとその一味が不老不死を求めて来襲し、ドラゴンボールを略奪しようとしていた。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空とナメック星人たちの協力を得てナメック星の神龍・ポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる。ウーロンがとっさに言い放った下らない願いを叶えてもらうことで、ピラフ一味がドラゴンボールを奪って神龍を呼び出そうとする野望を阻止する。3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。
DEBUG:root:> Refine context: 一方、魔人ブウは様々な人間との出会いからより邪悪で強力な魔人へと変貌。悟天とトランクスが「フュ...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["The original question is as follows: \\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=57718 request_id=6735c6aa4de1193c0a9ab56f775cacd0 response_code=200
DEBUG:root:

孫悟空とブルマは、実兄のラディッツから7つのドラゴンボールを集めることで神龍を呼び出すことができることを知り、それらを探す旅に出る。悟空はドラゴンボールによって生き返るまでの間、あの世の界王の下で修業し、仲間と共に地球に強襲したサイヤ人の戦士・ナッパと王子・ベジータを迎え撃つ。悟空は修術により増した力でナッパを一蹴し、ベジータと決闘。仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。その後、悟空は武術の達人・亀仙人の下で8か月間の修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟飯、クリリン、ブルマの3人が神とピッコロの故郷であるナメック星へ向かい、フリーザとその一味が不老不死を求めて来襲し、ドラゴンボールを略奪しようとしていた。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空とナメック星人たちの協力を得てナメック星の神龍・ポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる。ウーロンがとっさに言い放った下らない願いを叶えてもらうことで、ピラフ一味がドラゴンボールを奪って神龍を呼び出そうとする野望を阻止する。一方、魔人ブウは様々な人間との出会いからより邪悪で強力な魔人へと変貌。悟天とトランクスが「フュージョン（融合）」して誕生した戦士・ゴテンクスや、潜在能力を解放し、パワーアップを遂げて帰ってきた悟飯らが応戦するが、戦士たちを次々と吸収し姿を変えていく魔人ブウに苦戦を強いられる

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=69472 request_id=166eb529aa3591f50235fc3e99675bd7 response_code=200
DEBUG:root:

孫悟空とブルマは、実兄のラディッツから7つのドラゴンボールを集めることで神龍を呼び出すことができることを知り、それらを探す旅に出る。悟空はドラゴンボールによって生き返るまでの間、あの世の界王の下で修業し、仲間と共に地球に強襲したサイヤ人の戦士・ナッパと王子・ベジータを迎え撃つ。悟空は修術により増した力でナッパを一蹴し、ベジータと決闘。仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。その後、悟空は武術の達人・亀仙人の下で8か月間の修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟飯、クリリン、ブルマの3人が神とピッコロの故郷であるナメック星へ向かい、フリーザとその一味が不老不死を求めて来襲し、ドラゴンボールを略奪しようとしていた。悟飯たちはベジータ、フリーザ一味とのドラゴンボールをめぐる三つ巴の攻防の末、後から到着した悟空とナメック星人たちの協力を得てナメック星の神龍・ポルンガを呼び出し、ピッコロと地球のドラゴンボールを復活させる。ウーロンがとっさに言い放った下らない願いを叶えてもらうことで、ピラフ一味がドラゴンボールを奪って神龍を呼び出そうとする野望を阻止する。出し抜かれて願いが叶えられなかったフリーザは激怒し、一行は対決を強いられる。フリーザの持つ圧倒的な力の前にはベジータやピッコロ、悟空すら歯が立たず仲間たちが次々と命を落としていった。怒りを爆発させた悟空は伝説の戦士・超（スーパー）サイヤ人へと覚醒。フルパワーを解放したフリー

ValueError: A single term is larger than the allowed chunk size.
Term size: 323
Chunk size: 311Effective chunk size: 311

In [31]:
documents = SimpleDirectoryReader('story/dragonball').load_data()

#prompt_helper = PromptHelper(separator="\n\n")
# build index
index = GPTSimpleVectorIndex(documents)
response = index.query("ドラゴンボールのあらすじを要約してください", response_mode="tree_summarize", similarity_top_k=14)

DEBUG:root:> [SimpleDirectoryReader] Total files added: 1
DEBUG:root:> Adding chunk: 孫悟空少年編 地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た...
DEBUG:root:> Adding chunk:  魔人ブウ編 セルゲームより約7年後、高校生に成長した悟飯が天下一武道会に出場することを知った...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u5b6b\\u609f\\u7a7a\\u5c11\\u5e74\\u7de8 \\u5730\\u7403\\u306e\\u4eba\\u91cc\\u96e2\\u308c\\u305f\\u5c71\\u5965\\u306b\\u4f4f\\u3080\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u30fb\\u5b6b\\u609f\\u7a7a\\u306f\\u3042\\u308b\\u65e5\\u3001\\u897f\\u306e\\u90fd\\u304b\\u3089\\u3084\\u3063\\u3066\\u6765\\u305f\\u5c11\\u5973\\u30fb\\u30d6\\u30eb\\u30de\\u3068\\u51fa\\u4f1a\\u3046\\u3002\\u305d\\u3053\\u3067\\u30017\\u3064\\u96c6\\u3081\\u308b\\u3068\\u795e\\u9f8d\\uff08\\u30b7\\u30a7\\u30f3\\u30ed\\u30f3\\uff09\\u304c\\u73fe\\u308c\\u3001\\u3069\\u3093\\u306a\\u9858\\u3044\\u3067\\u3082\\u4

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 16697
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=232 request_id=5a09c3e9152c0b4af82a30a5bfde4e4d response_code=200
INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 4381 tokens
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u306e\\u3042\\u3089\\u3059\\u3058\\u3092\\u8981\\u7d04\\u3057\\u3066\\u304f\\u3060\\u3055\\u3044"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 8419
DEBUG:openai

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=13147 request_id=5184e738ce54a8a734089a86f99f652a response_code=200
DEBUG:root:
ドラゴンボールは、孫悟空がピッコロ大魔王、サイヤ人、フリーザ、そして人造人間・セルと戦い、地球を守る物語である。悟空は、界王神から恐ろしい力を持つ魔人ブウの封印が解かれようとしていることを知らされ、悟飯、クリリン、ブルマの3人が神とピッコロの故郷であるナメック星へ向かう。悟空は、修行により増した力でナッパを一蹴し、ベジータと決闘し、何とか辛勝し撤退させるが、多くの仲間を失うととも
DEBUG:root:> 0/9, summary: 
ドラゴンボールは、孫悟空がピッコロ大魔王、サイヤ人、フリーザ、そして人造人間・セルと戦い、地...
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Context information is below. \\n---------------------\\n\\n\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u306f\\u3001\\u5b6b\\u609f\\u7a7a\\u304c\\u30d4\\u30c3\\u30b3\\u30ed\\u5927\\u9b54\\u738b\\u3001\\u30b5\\u30a4\\u30e4\\u4eba\\u3001\\u30d5\\u30ea\\u30fc\\u30b6\\u3001\\u305d\\u3057\\u3066\\u4eb

In [40]:
max_input_size = 4096
num_output = 512
max_chunk_overlap = 0
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap, separator="\n\n", chunk_size_limit=512)

index = GPTSimpleVectorIndex(documents, prompt_helper=prompt_helper)
response = index.query("ドラゴンボールのあらすじを要約してください", response_mode="tree_summarize", similarity_top_k=14)

DEBUG:root:> Adding chunk: 孫悟空少年編
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た...
DEBUG:root:> Adding chunk: その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月...
DEBUG:root:> Adding chunk: ピッコロ大魔王編
天下一武道会終了後、ピラフ一味によって復活したピッコロ大魔王によって、クリリ...
DEBUG:root:> Adding chunk: その約3年後、少年から青年へと成長した悟空は、天下一武道会の会場で仲間たちと再会。試合では、少...
DEBUG:root:> Adding chunk: サイヤ人編
ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごし...
DEBUG:root:> Adding chunk: 悟空はドラゴンボールによって生き返るまでの間、あの世の界王の下で修業し、仲間と共に地球に強襲し...
DEBUG:root:> Adding chunk: フリーザ編
地球の神と殺された仲間たちを甦らせるため、重傷で入院中の悟空に代わり、悟飯、クリリ...
DEBUG:root:> Adding chunk: 出し抜かれて願いが叶えられなかったフリーザは激怒し、一行は対決を強いられる。フリーザの持つ圧倒...
DEBUG:root:> Adding chunk: 人造人間・セル編
ナメック星での闘いから約1年後、密かに生き延びていたフリーザとその一味が地球...
DEBUG:root:> Adding chunk: 人造人間17号と人造人間18号を吸収することで完全体となったセルは地球の命運を賭けた武道会「セ...
DEBUG:root:> Adding chunk: 魔人ブウ編
セルゲームより約7年後、高校生に成長した悟飯が天下一武道会に出場することを知った悟...
DEBUG:root:> Adding chunk: 一方、魔人ブウは様々な人間との出会いからより邪悪で強力な魔人へと変貌。悟天とトランクスが「フュ...
DEBUG:openai:message='Request to OpenAI API' method=post path=ht

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 82889
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=209 request_id=28424730c8a279408611d20426b0183b response_code=200
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u5b6b\\u609f\\u7a7a\\u5c11\\u5e74\\u7de8 \\u5730\\u7403\\u306e\\u4eba\\u91cc\\u96e2\\u308c\\u305f\\u5c71\\u5965\\u306b\\u4f4f\\u3080\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u30fb\\u5b6b\\u609f\\u7a7a\\u306f\\u3042\\u308b\\u65e5\\u3001\\u897f\\u306e\\u90fd\\u304b\\u3089\\u3084\\u3063\\u3066\\u6765\\u305f\\u5c11\\u5973\\u30fb\\u30d6\\u30eb\\u30de\\u3068\\u51fa\\u4f1a\\u3046\\u3002\\u305d\\u3053\\u3067\\u30017\\u3064\\u96c6\\u3081\\u308b\\u3068\\u795e\\u9f8d\\uff08\\u30b7\\u30a

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 429 285
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=15068 request_id=8dd52d1451caa14f9e00558fa3671455 response_code=429
INFO:openai:error_code=None error_message='The server is currently overloaded with other requests. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists.' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u5b6b\\u609f\\u7a7a\\u5c11\\u5e74\\u7de8 \\u5730\\u7403\\u306e\\u4eba\\u91cc\\u96e2\\u308c\\u305f\\u5c71\\u5965\\u306b\\u4f4f\\u3080\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 99439
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings processing_ms=302 request_id=9139211f5401269d21e4707583596c53 response_code=200
INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 4358 tokens
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-embedding-ada-002/embeddings
DEBUG:openai:api_version=None data='{"input": ["\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u306e\\u3042\\u3089\\u3059\\u3058\\u3092\\u8981\\u7d04\\u3057\\u3066\\u304f\\u3060\\u3055\\u3044"], "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-embedding-ada-002/embeddings HTTP/1.1" 200 8419
DEBUG:openai

ValueError: A single term is larger than the allowed chunk size.
Term size: 401
Chunk size: 352Effective chunk size: 352

In [45]:
from gpt_index import Document

text_list = doclist
documents = [Document(t) for t in text_list[0:7]]

In [49]:
index = GPTSimpleVectorIndex([])

# ドキュメントの挿入
for doc in documents:
    print(doc)
    index.insert(doc)

INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 0 tokens
Document(text='孫悟空少年編\n地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。', doc_id='eae3d73a-e5a1-4c26-9159-30c27adc159f', embedding=None, extra_info=None)


ValueError: Invalid document type: <class 'gpt_index.readers.schema.base.Document'>.

In [51]:
# なぜか動かない
# そもそもあのページは信用ならない...
# そもそもdocumentsがどういう構造をもっているのかもわかっていないのが課題である...
# 基本ができていないね
index = GPTSimpleVectorIndex(documents)
response = index.query("ドラゴンボールのあらすじを要約してください", response_mode="tree_summarize", similarity_top_k=7)

ValueError: Invalid document type: <class 'gpt_index.readers.schema.base.Document'>.

In [78]:
prompt = """
孫悟空少年編
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。
その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。

ピッコロ大魔王編
天下一武道会終了後、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。闘いの後、悟空はピッコロ大魔王に殺された神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。そこで神龍復活の条件として、神の下で天界で修行することとなった。

その約3年後、少年から青年へと成長した悟空は、天下一武道会の会場で仲間たちと再会。試合では、少年時代に出会った際に「嫁にもらう」と約束していた牛魔王の娘・チチと再会しその場で結婚。そして武道会に出場していたピッコロ大魔王の生まれ変わりであるマジュニアと決勝戦で激突、これに勝利し初の天下一武道会優勝を飾る。

サイヤ人編
ピッコロ（マジュニア）との闘いから約5年後、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空のもとに、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。さらわれた孫悟飯を助けるため悟空は宿敵ピッコロと手を組み、自らの命と引き換えにラディッツを倒すが、約1年後にはさらに強力なサイヤ人たちがドラゴンボールを求めて地球に来襲することを知る。

悟空はドラゴンボールによって生き返るまでの間、あの世の界王の下で修業し、仲間と共に地球に強襲したサイヤ人の戦士・ナッパと王子・ベジータを迎え撃つ。悟空は修行により増した力でナッパを一蹴し、ベジータと決闘。仲間の協力もあり、何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。

Tl;dr
ポイントは以下の3点です。
"""

In [80]:
prompt = """
「オントロジー」という用語は、哲学が起源であるが、多くの異なる意味で用いられる。単語『オントロジー』は、ギリシャ語で「存在」を意味する ὄν (on) から来ている。計算機科学では、タイプ、特性、および関係タイプのセットから成る世界を記述するためのモデルを指すことが多い。正確にこれらの変化の周りに準備されるものは様々だが、それらはオントロジーの必需品である。実世界と一つのオントロジーでのモデルの特徴との間の近い類似性が一般に期待されている[3]。 多くのオントロジーが計算機科学と哲学の両方で共通に持っているものは、エンティティ、アイデア、およびイベントに関する特性と関係とともに、それらをある分類システムに沿って表現することである。両方のフィールドで、存在論的相対性の問題（例えば、哲学ではQuineとKripke、計算機科学ではSowa（英語版）とGuarino（英語版））において重要な作業を見つけ出し、そして規範的（英語版）オントロジーが実行可能かどうかに関わる討議をする（例えば、哲学における基本[4]を議論し、AIにおけるCycプロジェクトを議論する）ことである。2つの間の相違は主として焦点の問題である。計算機科学者が第一次原則の議論（そのようなものが固定化された本質として存在するかどうか、あるいはエンティティがプロセスよりオントロジー的により一次的でなければならないかどうかのような議論）にあまり関わらない一方で、哲学者は計算機科学における研究者より固定化され統制された語彙を確立することにそれほど関心がない。哲学者は人間のための思考道具としてオントロジーを用いるのに対して、計算機科学者は計算機処理を考えるために人間が使う思考道具としてオントロジーを用いたいという立ち位置の違いによる。

Tl;dr
ポイントは以下の3点です。
"""

In [81]:
res = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    temperature=0,
    max_tokens=4096
)

print(res)
print(res.choices[0].text)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"model": "text-davinci-003", "prompt": "\\n\\u300c\\u30aa\\u30f3\\u30c8\\u30ed\\u30b8\\u30fc\\u300d\\u3068\\u3044\\u3046\\u7528\\u8a9e\\u306f\\u3001\\u54f2\\u5b66\\u304c\\u8d77\\u6e90\\u3067\\u3042\\u308b\\u304c\\u3001\\u591a\\u304f\\u306e\\u7570\\u306a\\u308b\\u610f\\u5473\\u3067\\u7528\\u3044\\u3089\\u308c\\u308b\\u3002\\u5358\\u8a9e\\u300e\\u30aa\\u30f3\\u30c8\\u30ed\\u30b8\\u30fc\\u300f\\u306f\\u3001\\u30ae\\u30ea\\u30b7\\u30e3\\u8a9e\\u3067\\u300c\\u5b58\\u5728\\u300d\\u3092\\u610f\\u5473\\u3059\\u308b \\u1f44\\u03bd (on) \\u304b\\u3089\\u6765\\u3066\\u3044\\u308b\\u3002\\u8a08\\u7b97\\u6a5f\\u79d1\\u5b66\\u3067\\u306f\\u3001\\u30bf\\u30a4\\u30d7\\u3001\\u7279\\u6027\\u3001\\u304a\\u3088\\u3073\\u95a2\\u4fc2\\u30bf\\u30a4\\u30d7\\u306e\\u30bb\\u30c3\\u30c8\\u304b\\u3089\\u6210\\u308b\\u4e16\\u754c\\u3092\\u8a18\\u8ff0\\u3059\\u308b\\u305f\\u3081\

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 5197 tokens (1101 in your prompt; 4096 for the completion). Please reduce your prompt; or completion length.

In [71]:
res = openai.Completion.create(
    model="text-davinci-003",
    prompt="初音ミクは",
    temperature=0,
    max_tokens=4096
)

print(res)
print(res.choices[0].text)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"model": "text-davinci-003", "prompt": "\\u521d\\u97f3\\u30df\\u30af\\u306f", "temperature": 0, "max_tokens": 4096}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 400 292
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=5 request_id=e737053e2845976864127d4710ec1ff4 response_code=400
INFO:openai:error_code=None error_message="This model's maximum context length is 4097 tokens, however you requested 4104 tokens (8 in your prompt; 4096 for the completion). Please reduce your prompt; or completion length." error_param=None error_type=invalid_request_error message='OpenAI API error received' stream_error=False


InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 4104 tokens (8 in your prompt; 4096 for the completion). Please reduce your prompt; or completion length.

In [75]:
prompt = """
初音ミクは
"""

res = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    max_tokens=200,
    temperature=0
)

print(res)
print(res.choices[0].text)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"model": "text-davinci-003", "prompt": "\\n\\u521d\\u97f3\\u30df\\u30af\\u306f\\n", "max_tokens": 200, "temperature": 0}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=9180 request_id=cc31ae872b2e1e5afa19c374de231234 response_code=200
{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\u521d\u97f3\u30df\u30af\u306f\u3001\u65e5\u672c\u306eVOCALOID\uff08\u30dc\u30fc\u30ab\u30ed\u30a4\u30c9\uff09\u30b7\u30ea\u30fc\u30ba\u306e\u30ad\u30e3\u30e9\u30af\u30bf\u30fc\u3067\u3059\u30022007\u5e74\u306bCrypton Future Media\u304b\u3089\u767a\u58f2\u3055\u308c\u305f\u521d\u306eVOCALOID\u3068\u3057\u3066\u767b\u5834\u3057\u3001\u4ee

In [82]:
import openai

def text_summary(prompt):
    # 分析の実施
    response = openai.Completion.create(
    engine="text-davinci-002",
    prompt=prompt,
    temperature=0.7,
    max_tokens=500,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    )

    # 分析結果の出力
    return response["choices"][0]["text"].replace('\n','')

def crean_text(text):
    text= text.replace('　',' ')

    return text


# 要約対象の文章をここに入力
text = '''
<input_text>
'''

# 要約を3点で出力するように調整
prompt ='''
Tl;dr
ポイントは以下の3点です。
'''

In [91]:
text="""
「オントロジー」という用語は、哲学が起源であるが、多くの異なる意味で用いられる。単語『オントロジー』は、ギリシャ語で「存在」を意味する ὄν (on) から来ている。計算機科学では、タイプ、特性、および関係タイプのセットから成る世界を記述するためのモデルを指すことが多い。正確にこれらの変化の周りに準備されるものは様々だが、それらはオントロジーの必需品である。実世界と一つのオントロジーでのモデルの特徴との間の近い類似性が一般に期待されている[3]。 多くのオントロジーが計算機科学と哲学の両方で共通に持っているものは、エンティティ、アイデア、およびイベントに関する特性と関係とともに、それらをある分類システムに沿って表現することである。両方のフィールドで、存在論的相対性の問題（例えば、哲学ではQuineとKripke、計算機科学ではSowa（英語版）とGuarino（英語版））において重要な作業を見つけ出し、そして規範的（英語版）オントロジーが実行可能かどうかに関わる討議をする（例えば、哲学における基本[4]を議論し、AIにおけるCycプロジェクトを議論する）ことである。2つの間の相違は主として焦点の問題である。計算機科学者が第一次原則の議論（そのようなものが固定化された本質として存在するかどうか、あるいはエンティティがプロセスよりオントロジー的により一次的でなければならないかどうかのような議論）にあまり関わらない一方で、哲学者は計算機科学における研究者より固定化され統制された語彙を確立することにそれほど関心がない。哲学者は人間のための思考道具としてオントロジーを用いるのに対して、計算機科学者は計算機処理を考えるために人間が使う思考道具としてオントロジーを用いたいという立ち位置の違いによる。
"""

In [84]:
prompt = crean_text(text) +  prompt


# 結果出力
print(text_summary(prompt))

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-davinci-002/completions
DEBUG:openai:api_version=None data='{"prompt": "\\n\\n\\u300c\\u30aa\\u30f3\\u30c8\\u30ed\\u30b8\\u30fc\\u300d\\u3068\\u3044\\u3046\\u7528\\u8a9e\\u306f\\u3001\\u54f2\\u5b66\\u304c\\u8d77\\u6e90\\u3067\\u3042\\u308b\\u304c\\u3001\\u591a\\u304f\\u306e\\u7570\\u306a\\u308b\\u610f\\u5473\\u3067\\u7528\\u3044\\u3089\\u308c\\u308b\\u3002\\u5358\\u8a9e\\u300e\\u30aa\\u30f3\\u30c8\\u30ed\\u30b8\\u30fc\\u300f\\u306f\\u3001\\u30ae\\u30ea\\u30b7\\u30e3\\u8a9e\\u3067\\u300c\\u5b58\\u5728\\u300d\\u3092\\u610f\\u5473\\u3059\\u308b \\u1f44\\u03bd (on) \\u304b\\u3089\\u6765\\u3066\\u3044\\u308b\\u3002\\u8a08\\u7b97\\u6a5f\\u79d1\\u5b66\\u3067\\u306f\\u3001\\u30bf\\u30a4\\u30d7\\u3001\\u7279\\u6027\\u3001\\u304a\\u3088\\u3073\\u95a2\\u4fc2\\u30bf\\u30a4\\u30d7\\u306e\\u30bb\\u30c3\\u30c8\\u304b\\u3089\\u6210\\u308b\\u4e16\\u754c\\u3092\\u8a18\\u8ff0\\u3059\\u308b\\u305f\\u3081\\

In [105]:
text="""
地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。その後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく。さらに3年後の天下一武道会では、亀仙流のライバルである鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。天下一武道会終了後、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、道中に出会ったヤジロベーや仙猫カリンの協力を得て命を賭して潜在する力を引き出し、ピッコロ大魔王に闘いを挑み勝利する。闘いの後、悟空はピッコロ大魔王に殺された神龍や仲間たちの復活のため天界へ向かい、ドラゴンボールの創造者である神に会う。そこで神龍復活の条件として、神の下で天界で修行することとなった。その約3年後、少年から青年へと成長した悟空は、天下一武道会の会場で仲間たちと再会。試合では、少年時代に出会った際に「嫁にもらう」と約束していた牛魔王の娘・チチと再会しその場で結婚。
"""

In [106]:
prompt = crean_text(text) +  prompt

# 結果出力
print(text_summary(prompt))

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/engines/text-davinci-002/completions
DEBUG:openai:api_version=None data='{"prompt": "\\n\\u5730\\u7403\\u306e\\u4eba\\u91cc\\u96e2\\u308c\\u305f\\u5c71\\u5965\\u306b\\u4f4f\\u3080\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u30fb\\u5b6b\\u609f\\u7a7a\\u306f\\u3042\\u308b\\u65e5\\u3001\\u897f\\u306e\\u90fd\\u304b\\u3089\\u3084\\u3063\\u3066\\u6765\\u305f\\u5c11\\u5973\\u30fb\\u30d6\\u30eb\\u30de\\u3068\\u51fa\\u4f1a\\u3046\\u3002\\u305d\\u3053\\u3067\\u30017\\u3064\\u96c6\\u3081\\u308b\\u3068\\u795e\\u9f8d\\uff08\\u30b7\\u30a7\\u30f3\\u30ed\\u30f3\\uff09\\u304c\\u73fe\\u308c\\u3001\\u3069\\u3093\\u306a\\u9858\\u3044\\u3067\\u3082\\u4e00\\u3064\\u3060\\u3051\\u53f6\\u3048\\u3066\\u304f\\u308c\\u308b\\u3068\\u3044\\u3046\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u306e\\u5b58\\u5728\\u3092\\u3001\\u3055\\u3089\\u306b\\u80b2\\u3066\\u306e\\u89aa\\u3067\\u3042\\u308b\\u5b6b\\u609f\\u9

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/engines/text-davinci-002/completions HTTP/1.1" 400 296
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/engines/text-davinci-002/completions processing_ms=196 request_id=41d0becabc447afc7e1bd250985c671e response_code=400
INFO:openai:error_code=None error_message="This model's maximum context length is 4097 tokens, however you requested 10743 tokens (10243 in your prompt; 500 for the completion). Please reduce your prompt; or completion length." error_param=None error_type=invalid_request_error message='OpenAI API error received' stream_error=False


InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 10743 tokens (10243 in your prompt; 500 for the completion). Please reduce your prompt; or completion length.

In [ ]:
# もうなんかいろいろおかしい